In [6]:
import chromadb
chroma_client = chromadb.Client()

In [7]:
collection = chroma_client.get_or_create_collection("my_collection")

In [8]:
collection.add(
    ids=["id1", "id2"],
    documents=[
        "This is a document about pineapple",
        "This is a document about oranges"
    ]
)

C:\Users\Kishore\.cache\chroma\onnx_models\all-MiniLM-L6-v2\onnx.tar.gz: 100%|██████████| 79.3M/79.3M [00:21<00:00, 3.90MiB/s]  


In [13]:
from pprint import pprint
results = collection.query(
    query_texts=["This is a query document about hawaii"], # Chroma will embed this for you
    n_results=2 # how many results to return
)
pprint(results)


{'data': None,
 'distances': [[1.0404009819030762, 1.2430799007415771]],
 'documents': [['This is a document about pineapple',
                'This is a document about oranges']],
 'embeddings': None,
 'ids': [['id1', 'id2']],
 'included': ['metadatas', 'documents', 'distances'],
 'metadatas': [[None, None]],
 'uris': None}


In [17]:
from pprint import pprint
results = collection.query(
    query_texts=["This is a query document about hawaii"], # Chroma will embed this for you
    n_results=2, # how many results to return
    where_document={'$contains': 'oranges'}
)
pprint(results)


{'data': None,
 'distances': [[1.2430799007415771]],
 'documents': [['This is a document about oranges']],
 'embeddings': None,
 'ids': [['id2']],
 'included': ['metadatas', 'documents', 'distances'],
 'metadatas': [[None]],
 'uris': None}


In [33]:
import polars as pl
articles = pl.read_csv("../data/articles.csv",encoding="ISO-8859-1").with_row_index(offset=1)
articles.head()

index,Article,Date,Heading,NewsType
u32,str,str,str,str
1,"""KARACHI: The Sindh government …","""1/1/2015""","""sindh govt decides to cut publ…","""business"""
2,"""HONG KONG: Asian markets start…","""1/2/2015""","""asia stocks up in new year tra…","""business"""
3,"""HONG KONG: Hong Kong shares o…","""1/5/2015""","""hong kong stocks open 0.66 per…","""business"""
4,"""HONG KONG: Asian markets tumbl…","""1/6/2015""","""asian stocks sink euro near ni…","""business"""
5,"""NEW YORK: US oil prices Monday…","""1/6/2015""","""us oil prices slip below 50 a …","""business"""
